## **Reviews Scraping**

In [2]:
!pip install google-play-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

import pandas as pd
import numpy as np

In [4]:
reviews = reviews_all(
    'gov.fema.mobile.android',
    country = 'us',
    sort=Sort.NEWEST
)

In [5]:
df = pd.DataFrame(np.array(reviews),columns=['review'])
df = df.join(pd.DataFrame(df.pop('review').tolist()))

In [6]:
df = df.rename(columns = {'content' : 'review',
                     'score' : 'rating',
                     'thumbsUpCount' : 'helpfulCount',
                     'at' : 'date' })

In [7]:
df.to_csv("reviews.csv",index=False)

## **Libraries**

In [8]:
!pip install emoji
!pip install contractions
!pip install pyLDAvis==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=187369a71f43a2f27cf3d319344627ae765af14e02023c6db14876ea422fa6d6
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel fo

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
import re
import spacy
import pprint
from nltk.corpus import wordnet
import nltk.collocations 
from textblob import TextBlob
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from contractions import contractions_dict
from textblob import TextBlob
from PIL import Image
from emoji import demojize 


import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import gensim
from gensim import corpora

import warnings
warnings.filterwarnings("ignore")

import pyLDAvis
import pyLDAvis.gensim
from gensim.models import CoherenceModel

from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## **Data Pre-Processing**

In [11]:
df = pd.read_csv("/content/reviews.csv")

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df.head(3)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,reviewId,userName,userImage,review,rating,helpfulCount,reviewCreatedVersion,date,replyContent,repliedAt
0,e3dd3b73-3755-402c-9697-de3daab1fe24,G,https://play-lh.googleusercontent.com/a/AGNmyx...,"Excellent, helpful app!",5,0,3.0.11,2023-04-01 09:36:31,NaN,NaN
1,7c0b100c-6978-4863-b138-9c8b82a7be3e,Stacey Vue-Lo,https://play-lh.googleusercontent.com/a-/ACB-R...,LOVING IT!!,5,0,3.0.11,2023-03-29 20:22:57,NaN,NaN
2,c98072b7-3603-4c1c-809a-4192bc830896,Mark Demo,https://play-lh.googleusercontent.com/a-/ACB-R...,It's the Federal Government.,5,0,3.0.11,2023-03-28 04:06:46,NaN,NaN


In [13]:
df1 = df[df['helpfulCount']!=0]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
df1.shape

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(775, 10)

In [15]:
df1.to_csv("reviews1.csv",index=False)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
